In [1]:
from hyperband.hyperband import Hyperband
from hyperopt import hp
from xgboost import XGBRegressor as XGB
from hyperopt.pyll.stochastic import sample
import datetime
import pandas as pd
import numpy as np
from pprint import pprint
import json
from sklearn.gaussian_process import kernels, GaussianProcessRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats
from functools import reduce

In [2]:
def load_data():
    dframe_path = 'data/cabauw/processed-full-log.csv.gz'
    df = pd.read_csv(dframe_path, na_values='--', compression='gzip')

    df = df[(df.ustar > 0.1) & (abs(df.H) > 10) & (df.wind > 1)]
    df = df[df.ds != 201603]
    
    return df

df = load_data()
wind_temp_levels = df.pivot_table(
    values=['wind', 'temp'], columns='z', index=['ds', 'tt']
).reset_index()
wind_temp_levels.columns = [
    '%s_%d' % (a, b) if b else a
    for a, b in wind_temp_levels.columns.values
]

ddf = df.merge(wind_temp_levels, on=['ds', 'tt']).dropna()
ddf = ddf[(ddf.zL > -2) & (ddf.zL < 1)]

feature_sets = [
    [
        'z', 'wind', 'temp', 'soil_temp',
        'wind_10', 'wind_20', 'wind_40',
        'temp_10', 'temp_20', 'temp_40',
    ],
    ['soilheat'],
    ['netrad'],
    ['rain', 'dewpoint'],
]

feats = [f for fset in feature_sets for f in fset]

In [3]:
np.random.seed(948235697)

test_ds = np.random.choice(ddf.ds.unique(), 20, replace=False)
test_mask = ddf.ds.isin(test_ds)

train_x, train_y = ddf[~test_mask][feats].dropna(), ddf[~test_mask].phi_m
test_x, test_y = ddf[test_mask][feats].dropna(), ddf[test_mask].phi_m

mu_x, std_x = train_x.mean(), train_x.std()
mu_y, std_y = train_y.mean(), train_y.std()

train_x = (train_x - mu_x) / std_x
test_x = (test_x - mu_x) / std_x

train_y = (train_y - mu_y) / std_y
test_y = (test_y - mu_y) / std_y

In [4]:
space = {
    'learning_rate': hp.loguniform('ra_', np.log(1e-4), np.log(1)),
    'max_depth': hp.quniform('md_', 2, 12, 1),
    'min_child_weight': hp.quniform('mcw_', 1, 10, 1),
    'subsample': hp.uniform('ss_', 0.25, 1.0),
    'gamma': hp.uniform('g_', 0, 1),
    'reg_alpha': hp.loguniform('ra_', np.log(1e-10), np.log(1)),
    'reg_lambda': hp.loguniform('ra_', np.log(1e-10), np.log(1)),
    'colsample_bytree': hp.uniform('cbt_', 0.25, 1.0),
    'loss': hp.choice('loss', ['rmse', 'mae',]),
}


def handle_integers(params):
    new_params = {}
    for k, v in params.items():
        if type(v) == float and int(v) == v:
            new_params[k] = int(v)
        else:
            new_params[k] = v

    return new_params


def get_params():
    params = sample(space)
    params = {k: v for k, v in params.items() if v is not 'default'}
    return handle_integers(params)


trees_per_iteration = 10
def try_params(n_iterations, params, get_predictions=False):
    params['n_estimators'] = int(round(n_iterations * trees_per_iteration))

    result = fit_score(params, get_predictions)
    return result


def fit_score(params, get_predictions=False, save=True):
    try:
        model = XGB(tree_method='gpu_exact', **params)
        model.fit(train_x, train_y)
        y_pred = model.predict(test_x)
    finally:
        del(model)  # https://github.com/dmlc/xgboost/issues/3045

    mse = np.mean((y_pred - test_y)**2)    
    denorm = mse * std_y**2

    result = {
        'mse': mse,
        'loss': denorm,
        'denorm': denorm,
        'preds': preds if get_predictions else None,
        'params': params,
    }
    
    if save:
        with open('dev/hyperopt.jsonl', 'a') as f:
            f.write(json.dumps(result))
            f.write('\n')

    return result

hyper parameters optimization with hyperband



paper https://arxiv.org/pdf/1603.06560.pdf

code https://github.com/zygmuntz/hyperband

In [9]:
hb = Hyperband(get_params, try_params)
hb.run()


*** 81 configurations x 1.0 iterations each

1 | Fri May 18 20:06:13 2018 | lowest loss so far: inf (run -1)

0.8724171336225047 {'colsample_bytree': 0.5785577454075457, 'gamma': 0.7239266100837887, 'learning_rate': 0.020578037327198984, 'loss': 'rmse', 'max_depth': 2, 'min_child_weight': 6, 'reg_alpha': 0.029963754672958868, 'reg_lambda': 7.584990990738952e-05, 'subsample': 0.4170265496831048, 'n_estimators': 10}

4 seconds.

2 | Fri May 18 20:06:17 2018 | lowest loss so far: 0.8724 (run 1)

1.0389289600246219 {'colsample_bytree': 0.9420708081773133, 'gamma': 0.0913100403132221, 'learning_rate': 0.0005017338862024305, 'loss': 'rmse', 'max_depth': 10, 'min_child_weight': 7, 'reg_alpha': 0.0017742130053565461, 'reg_lambda': 0.8673959619924567, 'subsample': 0.444889514645635, 'n_estimators': 10}

7 seconds.

3 | Fri May 18 20:06:25 2018 | lowest loss so far: 0.8724 (run 1)

0.4670748634633622 {'colsample_bytree': 0.6380487363455003, 'gamma': 0.5796512869574597, 'learning_rate': 0.087307

0.22091944566899988 {'colsample_bytree': 0.9164796249297884, 'gamma': 0.48187927854549695, 'learning_rate': 0.1796974701392746, 'loss': 'rmse', 'max_depth': 10, 'min_child_weight': 4, 'reg_alpha': 1.0679822847971537e-08, 'reg_lambda': 1.0700995952034202e-10, 'subsample': 0.3960565625465873, 'n_estimators': 10}

11 seconds.

23 | Fri May 18 20:08:35 2018 | lowest loss so far: 0.1968 (run 13)

1.0270840508752748 {'colsample_bytree': 0.7462492691874866, 'gamma': 0.07472229834603261, 'learning_rate': 0.0012327078420383318, 'loss': 'rmse', 'max_depth': 11, 'min_child_weight': 5, 'reg_alpha': 7.181778011876671e-08, 'reg_lambda': 4.858540286427487e-07, 'subsample': 0.6384177019223621, 'n_estimators': 10}

7 seconds.

24 | Fri May 18 20:08:42 2018 | lowest loss so far: 0.1968 (run 13)

1.0438452324527112 {'colsample_bytree': 0.45985034461077834, 'gamma': 0.8211339427844362, 'learning_rate': 0.0001905279873914263, 'loss': 'rmse', 'max_depth': 10, 'min_child_weight': 7, 'reg_alpha': 1.2841286045

1.0436907165580247 {'colsample_bytree': 0.6261792750522378, 'gamma': 0.25801346892402066, 'learning_rate': 0.0002600764169096858, 'loss': 'rmse', 'max_depth': 4, 'min_child_weight': 10, 'reg_alpha': 7.932866625721244e-10, 'reg_lambda': 2.055519200844954e-10, 'subsample': 0.8101977489348469, 'n_estimators': 10}

4 seconds.

44 | Fri May 18 20:10:43 2018 | lowest loss so far: 0.1968 (run 13)

1.0355276381276168 {'colsample_bytree': 0.603141975946395, 'gamma': 0.12250253631246832, 'learning_rate': 0.0009345560413348185, 'loss': 'mae', 'max_depth': 4, 'min_child_weight': 9, 'reg_alpha': 0.49556457279544025, 'reg_lambda': 0.0007146782325731805, 'subsample': 0.6226652141385363, 'n_estimators': 10}

4 seconds.

45 | Fri May 18 20:10:47 2018 | lowest loss so far: 0.1968 (run 13)

1.039782016257109 {'colsample_bytree': 0.9295982605357749, 'gamma': 0.6807493651130845, 'learning_rate': 0.0004973223682512474, 'loss': 'rmse', 'max_depth': 7, 'min_child_weight': 9, 'reg_alpha': 0.01414495245160964, 

0.9312101434382133 {'colsample_bytree': 0.6723940887817069, 'gamma': 0.027744091094098366, 'learning_rate': 0.009714380603340454, 'loss': 'mae', 'max_depth': 5, 'min_child_weight': 6, 'reg_alpha': 3.2723029198371895e-09, 'reg_lambda': 1.2648615113151112e-05, 'subsample': 0.29442330960471896, 'n_estimators': 10}

5 seconds.

65 | Fri May 18 20:12:37 2018 | lowest loss so far: 0.1968 (run 13)

0.5611580262566762 {'colsample_bytree': 0.4114618834187501, 'gamma': 0.9403289283949885, 'learning_rate': 0.05604344988371662, 'loss': 'rmse', 'max_depth': 6, 'min_child_weight': 6, 'reg_alpha': 7.293342644470763e-08, 'reg_lambda': 3.069377659447525e-10, 'subsample': 0.4768452421301991, 'n_estimators': 10}

5 seconds.

66 | Fri May 18 20:12:42 2018 | lowest loss so far: 0.1968 (run 13)

1.0415977805443006 {'colsample_bytree': 0.35757254786883064, 'gamma': 0.5740522730657681, 'learning_rate': 0.0003200979803074287, 'loss': 'rmse', 'max_depth': 12, 'min_child_weight': 5, 'reg_alpha': 1.94020921542756

1.5541999690539408 {'colsample_bytree': 0.9597007542224114, 'gamma': 0.20950651754134886, 'learning_rate': 0.1898406677399128, 'loss': 'rmse', 'max_depth': 12, 'min_child_weight': 5, 'reg_alpha': 8.801804491805221e-05, 'reg_lambda': 0.011253079315783737, 'subsample': 0.5808373440729613, 'n_estimators': 30}

17 seconds.

86 | Fri May 18 20:15:13 2018 | lowest loss so far: 0.1968 (run 13)

0.2014687688814392 {'colsample_bytree': 0.6470018847464235, 'gamma': 0.4310103093038139, 'learning_rate': 0.6153448590137603, 'loss': 'mae', 'max_depth': 7, 'min_child_weight': 3, 'reg_alpha': 3.103540630974797e-05, 'reg_lambda': 0.004663678664804135, 'subsample': 0.4221400416627633, 'n_estimators': 30}

11 seconds.

87 | Fri May 18 20:15:24 2018 | lowest loss so far: 0.1968 (run 13)

0.18292992562389435 {'colsample_bytree': 0.9164796249297884, 'gamma': 0.48187927854549695, 'learning_rate': 0.1796974701392746, 'loss': 'rmse', 'max_depth': 10, 'min_child_weight': 4, 'reg_alpha': 1.0679822847971537e-08, 

...

0.8170529630201893 {'colsample_bytree': 0.7333191907796599, 'gamma': 0.24722763151107519, 'learning_rate': 0.006342624604266202, 'loss': 'rmse', 'max_depth': 7, 'min_child_weight': 8, 'reg_alpha': 0.0018754297009490007, 'reg_lambda': 6.509624653957135e-06, 'subsample': 0.3131477961361745, 'n_estimators': 30}

8 seconds.

128 | Fri May 18 20:27:44 2018 | lowest loss so far: 0.1788 (run 118)

0.2189594421494199 {'colsample_bytree': 0.7707173745437496, 'gamma': 0.9064339254027001, 'learning_rate': 0.8161097994239517, 'loss': 'mae', 'max_depth': 4, 'min_child_weight': 9, 'reg_alpha': 4.3928094267033936e-07, 'reg_lambda': 1.6775454569003994e-07, 'subsample': 0.47729606339040465, 'n_estimators': 30}

6 seconds.

129 | Fri May 18 20:27:50 2018 | lowest loss so far: 0.1788 (run 118)

1.0311273830921417 {'colsample_bytree': 0.7745703253836151, 'gamma': 0.07760312867038921, 'learning_rate': 0.0004341174874963597, 'loss': 'mae', 'max_depth': 4, 'min_child_weight': 5, 'reg_alpha': 5.59894369726371

0.913197139205766 {'colsample_bytree': 0.5620968935128592, 'gamma': 0.3294271886219857, 'learning_rate': 0.003275348251835255, 'loss': 'rmse', 'max_depth': 8, 'min_child_weight': 5, 'reg_alpha': 1.7971664739550668e-09, 'reg_lambda': 6.275316344632771e-05, 'subsample': 0.9961639665268283, 'n_estimators': 30}

9 seconds.

149 | Fri May 18 20:30:21 2018 | lowest loss so far: 0.1788 (run 118)

0.24265341731312703 {'colsample_bytree': 0.5766513537443907, 'gamma': 0.4550853475264506, 'learning_rate': 0.07945861733534562, 'loss': 'mae', 'max_depth': 7, 'min_child_weight': 9, 'reg_alpha': 3.1614501756056725e-06, 'reg_lambda': 2.323991688296225e-06, 'subsample': 0.8634198722332566, 'n_estimators': 30}

8 seconds.

150 | Fri May 18 20:30:29 2018 | lowest loss so far: 0.1788 (run 118)

0.2076177488719325 {'colsample_bytree': 0.7345930083903538, 'gamma': 0.176325905860791, 'learning_rate': 0.42280173521819914, 'loss': 'mae', 'max_depth': 5, 'min_child_weight': 6, 'reg_alpha': 2.6693066423301315e-0

0.1773808415415551 {'colsample_bytree': 0.5766513537443907, 'gamma': 0.4550853475264506, 'learning_rate': 0.07945861733534562, 'loss': 'mae', 'max_depth': 7, 'min_child_weight': 9, 'reg_alpha': 3.1614501756056725e-06, 'reg_lambda': 2.323991688296225e-06, 'subsample': 0.8634198722332566, 'n_estimators': 270}

55 seconds.

*** 1.259259259259259 configurations x 81.0 iterations each

170 | Fri May 18 20:37:14 2018 | lowest loss so far: 0.1774 (run 169)

0.1743771648176164 {'colsample_bytree': 0.5766513537443907, 'gamma': 0.4550853475264506, 'learning_rate': 0.07945861733534562, 'loss': 'mae', 'max_depth': 7, 'min_child_weight': 9, 'reg_alpha': 3.1614501756056725e-06, 'reg_lambda': 2.323991688296225e-06, 'subsample': 0.8634198722332566, 'n_estimators': 810}

159 seconds.

*** 15 configurations x 9.0 iterations each

171 | Fri May 18 20:39:53 2018 | lowest loss so far: 0.1744 (run 170)

0.18724847632239772 {'colsample_bytree': 0.34648813790510524, 'gamma': 0.7675251886272555, 'learning_rate

0.22454718348972758 {'colsample_bytree': 0.2764407807836974, 'gamma': 0.3452635587614632, 'learning_rate': 0.8146268118161465, 'loss': 'rmse', 'max_depth': 8, 'min_child_weight': 5, 'reg_alpha': 3.3467015449357943e-07, 'reg_lambda': 2.156953948307795e-06, 'subsample': 0.7803142790554474, 'n_estimators': 270}

59 seconds.

*** 1.6666666666666665 configurations x 81.0 iterations each

191 | Fri May 18 20:49:17 2018 | lowest loss so far: 0.1744 (run 170)

54.185358642765216 {'colsample_bytree': 0.798679454767544, 'gamma': 0.2225296465976453, 'learning_rate': 0.22668943626469515, 'loss': 'mae', 'max_depth': 7, 'min_child_weight': 4, 'reg_alpha': 0.0016739519821775618, 'reg_lambda': 0.0031159078552678294, 'subsample': 0.7587491156596674, 'n_estimators': 810}

153 seconds.

*** 8 configurations x 27.0 iterations each

192 | Fri May 18 20:51:50 2018 | lowest loss so far: 0.1744 (run 170)

0.33888428138446364 {'colsample_bytree': 0.3450104692494317, 'gamma': 0.20171901979350204, 'learning_rate

[{'mse': 1.041761877672452,
  'loss': 0.8724171336225047,
  'denorm': 0.8724171336225047,
  'preds': None,
  'params': {'colsample_bytree': 0.5785577454075457,
   'gamma': 0.7239266100837887,
   'learning_rate': 0.020578037327198984,
   'loss': 'rmse',
   'max_depth': 2,
   'min_child_weight': 6,
   'reg_alpha': 0.029963754672958868,
   'reg_lambda': 7.584990990738952e-05,
   'subsample': 0.4170265496831048,
   'n_estimators': 10},
  'counter': 1,
  'seconds': 4,
  'iterations': 1.0},
 {'mse': 1.2405952868778214,
  'loss': 1.0389289600246219,
  'denorm': 1.0389289600246219,
  'preds': None,
  'params': {'colsample_bytree': 0.9420708081773133,
   'gamma': 0.0913100403132221,
   'learning_rate': 0.0005017338862024305,
   'loss': 'rmse',
   'max_depth': 10,
   'min_child_weight': 7,
   'reg_alpha': 0.0017742130053565461,
   'reg_lambda': 0.8673959619924567,
   'subsample': 0.444889514645635,
   'n_estimators': 10},
  'counter': 2,
  'seconds': 7,
  'iterations': 1.0},
 {'mse': 0.557738687

best is 0.17



disappointing

lets try something more traditional

In [5]:
import GPy
import GPyOpt

In [6]:
log10columns = {'learning_rate', 'reg_alpha', 'reg_lambda', 'n_estimators'}


def read_results():
    res = []
    with open('dev/hyperopt.jsonl') as f:
        for row in f:
            evals = json.loads(row)
            params = evals['params']
            for logcol in log10columns:
                params[logcol] = np.log10(params[logcol])
            
            params['denorm'] = evals['denorm']
            res.append(params)

    res = pd.DataFrame(res)
    return res


res = read_results()
res.sample(5)

,colsample_bytree,denorm,gamma,learning_rate,loss,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample
350,0.573539,407.638465,0.493343,-0.257715,rmse,10,10,3.000000,-5.205077,-20.000000,0.342250
19,0.746249,1.027084,0.074722,-2.909140,rmse,11,5,1.000000,-7.143768,-6.313494,0.638418
22,0.360375,0.415982,0.370609,-1.068529,mae,7,4,1.000000,-1.147595,-9.268654,0.919828
117,0.682832,806.156046,0.989058,-0.709571,mae,8,3,2.908485,-6.211880,-4.569266,0.505705
306,0.792638,0.975360,0.749330,-5.014462,rmse,3,5,3.000000,-20.000000,-20.000000,0.330236


In [7]:
# bounds for GP optimization
bounds = [dict(zip(['name', 'type', 'domain'], spec)) for spec in [
    ('n_estimators', 'continuous', (1, 3)),
    ('max_depth', 'discrete', (1, 12)),
    ('min_child_weight', 'discrete', (1, 100)),
    ('learning_rate', 'continuous', (-6, -1)),
    ('reg_alpha', 'continuous', (-10, 0)),
    ('reg_lambda', 'continuous', (-10, 0)),
    ('loss', 'discrete', (0, 1)),
    ('colsample_bytree', 'continuous', (0.2, 1.0)),
    ('subsample', 'continuous', (0.2, 1.0)),
    ('gamma', 'continuous', (0, 1)),
]]


# only keep results within these bounds
def is_within(bb):
    if bb['name'] == 'loss':
        return np.array([True] * len(res))

    vals = res[bb['name']]
    lo, hi = bb['domain']
    return (vals >= lo) & (vals <= hi)


all_within = reduce(np.logical_and, map(is_within, bounds))
res = res[all_within & (res.denorm < 5)]
res.loss = res.loss.map(lambda x: {'rmse': 0, 'mae': 1}.get(x, x))
len(res)

136

In [22]:
keys = [bb['name'] for bb in bounds]
def sanitize_params(pp):
    hyp = dict(zip(keys, pp))
    hyp['n_estimators'] = int(round(10**hyp['n_estimators']))
    hyp['max_depth'] = int(round(hyp['max_depth']))
    hyp['min_child_weight'] = int(round(hyp['min_child_weight']))
    hyp['learning_rate'] = 10**hyp['learning_rate']
    hyp['reg_alpha'] = 10**hyp['reg_alpha']
    hyp['reg_lambda'] = 10**hyp['reg_lambda']
    hyp['loss'] = ['rmse', 'mae'][int(hyp['loss'])]
    hyp['gamma'] = max(0, hyp['gamma'])

    return hyp


class TargetFunction:
    def __init__(self):
        self.best = np.inf
        self.count = 0
    
    def log(self, result):
        self.best = min(self.best, result['denorm'])
        self.count += 1
        
        dtime = datetime.datetime.now().isoformat().replace('T', ' ')[:-7]
        print('%d | %s | Got %.4f | Best so far: %.4f' % (
            self.count, dtime, result['denorm'], self.best
        ))
    
    def __call__(self, params):
        res = np.zeros((params.shape[0], 1))
        for i, pp in enumerate(params):
            hyp = sanitize_params(pp)
            result = fit_score(hyp, save=True)
            result['denorm'] = min(10, result['denorm'])
            res[i] = result['denorm']
            self.log(result)
        return res

In [23]:
target = TargetFunction()
bopt = GPyOpt.methods.BayesianOptimization(
    target, bounds,
    model_type='GP_MCMC',
    acquisition_type='EI_MCMC'
    # can use old results to speed this up, but where's the fun then
    #X=res.drop(['denorm'], axis=1).values,
    #Y=res.denorm.values.reshape(-1, 1),
)

bopt.run_optimization(max_iter=1000)
bopt.fx_opt

1 | 2018-05-22 12:28:13 | Got 0.9304 | Best so far: 0.9304
2 | 2018-05-22 12:28:16 | Got 0.9321 | Best so far: 0.9304
3 | 2018-05-22 12:28:25 | Got 0.9809 | Best so far: 0.9304
4 | 2018-05-22 12:28:40 | Got 0.7296 | Best so far: 0.7296


reconstraining parameters GP_regression.rbf
reconstraining parameters GP_regression.Gaussian_noise.variance


5 | 2018-05-22 12:30:08 | Got 0.5782 | Best so far: 0.5782
6 | 2018-05-22 12:32:01 | Got 0.5782 | Best so far: 0.5782
7 | 2018-05-22 12:34:44 | Got 0.3410 | Best so far: 0.3410
8 | 2018-05-22 12:40:29 | Got 0.1839 | Best so far: 0.1839
9 | 2018-05-22 12:46:32 | Got 0.1794 | Best so far: 0.1794
10 | 2018-05-22 12:52:42 | Got 0.1860 | Best so far: 0.1794
11 | 2018-05-22 12:58:42 | Got 0.1786 | Best so far: 0.1786
12 | 2018-05-22 13:04:16 | Got 0.1784 | Best so far: 0.1784
13 | 2018-05-22 13:05:59 | Got 0.1876 | Best so far: 0.1784
14 | 2018-05-22 13:12:38 | Got 0.9843 | Best so far: 0.1784
15 | 2018-05-22 13:18:31 | Got 0.1784 | Best so far: 0.1784
16 | 2018-05-22 13:24:10 | Got 0.1805 | Best so far: 0.1784
17 | 2018-05-22 13:29:44 | Got 0.2936 | Best so far: 0.1784
18 | 2018-05-22 13:30:03 | Got 6.3680 | Best so far: 0.1784
19 | 2018-05-22 13:31:05 | Got 0.9851 | Best so far: 0.1784
20 | 2018-05-22 13:37:48 | Got 0.9843 | Best so far: 0.1784
21 | 2018-05-22 13:39:03 | Got 0.9851 | Best 

array([0.1729335])

In [29]:
sanitize_params(bopt.x_opt)

{'n_estimators': 1000,
 'max_depth': 12,
 'min_child_weight': 100,
 'learning_rate': 0.1,
 'reg_alpha': 1e-10,
 'reg_lambda': 1e-10,
 'loss': 'mae',
 'colsample_bytree': 0.19999999999999998,
 'subsample': 0.2,
 'gamma': 0}